<a href="https://colab.research.google.com/github/jalew188/PeptDeep-HLA/blob/master/nbs/HLA1_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning of sample-specific HLA-I models

> To enable GPU in colab, click `Runtime -> Change runtime type`.

In [ ]:
%pip install -q git+https://github.com/MannLabs/PeptDeep-HLA.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.0/554.0 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install -q gdown

In [ ]:
import torch
if torch.cuda.is_available():
  # human fasta
  fasta_url = "https://drive.google.com/file/d/1V9KxDniKwZFZnHlP58EbjkuelNJnp1Kq/view?usp=share_link"
  fasta = 'UP000005640_human_reviewed.fasta'
else:
  # no GPU runtime in colab, use irt fusion peptides for testing
  fasta_url = "https://drive.google.com/file/d/1MKGRBpzvmMW0l_hdPESo3j26EWd_yi8l/view?usp=share_link"
  fasta = 'irtfusion.fasta'

In [ ]:
import gdown

gdown.download(fasta_url, fasta, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1V9KxDniKwZFZnHlP58EbjkuelNJnp1Kq
To: /content/UP000005640_human_reviewed.fasta
100%|██████████| 13.6M/13.6M [00:00<00:00, 26.1MB/s]


'UP000005640_human_reviewed.fasta'

#### Load training HLA peptides

It can be a tsv/csv/txt file containing sample-specific HLA-I peptides in the 'sequence' column.

Click the `Files` (folder logo) in the left panel of Colab and upload files.

In [ ]:
the_file_you_uploaded = ""

In [ ]:
import pandas as pd

if the_file_you_uploaded:
  train_seq_df = pd.read_csv(the_file_you_uploaded, sep='\t')
else:
  train_seq_df = pd.DataFrame({
    'sequence': [
        'ACDEFGHIKLMNPQ',
        'ACDEFGHI',
        'ACDEFGHIK',
        'EFGHIKLMNPQ',
        'AHIKLMNPQ',
    ]
  })
train_seq_df['nAA'] = train_seq_df.sequence.str.len()
train_seq_df

,sequence,nAA
0,ACDEFGHIKLMNPQ,14
1,ACDEFGHI,8
2,ACDEFGHIK,9
3,EFGHIKLMNPQ,11
4,AHIKLMNPQ,9


#### Initialize the model and load the pretrained model

In [ ]:
from peptdeep_hla.HLA_class_I import HLA_Class_I_Classifier
model = HLA_Class_I_Classifier(
    fasta_files=[fasta]
)
model.get_parameter_num()

2023-01-03 20:31:48> Downloading pretrained_models.zip ...
2023-01-03 20:31:51> The pretrained models had been downloaded in /root/peptdeep/pretrained_models/pretrained_models.zip


1669697

In [ ]:
from peptdeep_hla.HLA_class_I import pretrained_HLA1
model.load(pretrained_HLA1)
pretrained_HLA1

'/usr/local/lib/python3.8/dist-packages/peptdeep_hla/pretrained_models/HLA1_IEDB.pt'

#### Train by the training peptides

The non-HLA peptides are automatically sampled from the fasta file as the negative training data.

In [ ]:
model.train(
    train_seq_df, 
    epoch=20, warmup_epoch=10, 
    verbose=True
)

[Training] Epoch=1, lr=1e-05, loss=1.4590696841478348
[Training] Epoch=2, lr=2e-05, loss=1.4447193890810013
[Training] Epoch=3, lr=3e-05, loss=1.4153653532266617
[Training] Epoch=4, lr=4e-05, loss=0.7665752694010735
[Training] Epoch=5, lr=5e-05, loss=0.873346634209156
[Training] Epoch=6, lr=6e-05, loss=0.5855524465441704
[Training] Epoch=7, lr=7e-05, loss=0.18241026252508163
[Training] Epoch=8, lr=8e-05, loss=0.3510209955275059
[Training] Epoch=9, lr=9e-05, loss=0.49503829004243016
[Training] Epoch=10, lr=0.0001, loss=0.11926002241671085
[Training] Epoch=11, lr=9.755282581475769e-05, loss=0.30639077723026276
[Training] Epoch=12, lr=9.045084971874738e-05, loss=0.47722677886486053
[Training] Epoch=13, lr=7.938926261462366e-05, loss=0.2694461327046156
[Training] Epoch=14, lr=6.545084971874738e-05, loss=0.01919158105738461
[Training] Epoch=15, lr=5e-05, loss=0.017440842115320265
[Training] Epoch=16, lr=3.4549150281252636e-05, loss=0.01759017980657518
[Training] Epoch=17, lr=2.0610737385376

#### Predict HLA-I peptides from fasta

In [ ]:
hla_df = model.predict_from_proteins(prob_threshold=0.7)
hla_df

/usr/local/lib/python3.8/dist-packages/pydivsufsort/divsufsort.py:76: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")
/usr/local/lib/python3.8/dist-packages/alphabase/protein/lcp_digest.py:7: UserWarning: converting str argument uses more memory
  lcp_array = kasai(cat_prot, suffix_array)
100%|██████████| 72/72 [56:22<00:00, 46.99s/it]


,start_pos,end_pos,nAA,HLA_prob_pred,sequence
0,4709883,4709891,8,0.853650,TESEARAL
1,4709854,4709862,8,0.769178,LPPPGLTI
2,4709869,4709877,8,0.758194,LPNIKREL
3,4709810,4709818,8,0.968220,VIDDIISL
4,4710441,4710449,8,0.932903,AEAPPPQI
...,...,...,...,...,...
849954,7595702,7595716,14,0.792721,HHDYLLGHLDDEGK
849955,9899415,9899429,14,0.947694,KRFEFPNSLIQSQA
849956,2563499,2563513,14,0.878654,DTVPSSTNQVVGSR
849957,7595719,7595733,14,0.988616,QSDRGKGAEGLGTY


In [ ]:
hla_df[['sequence','HLA_prob_pred']].to_csv('Predicted_HLA.tsv',index=False, sep="\t")

To download `Predicted_HLA.tsv` when using Colab, click the `Files` (folder logo) in the left panel and right-click the file to download.

In [ ]:
#@title Download Predicted_HLA.tsv
from google.colab import files
files.download(f'Predicted_HLA.tsv')

In [ ]:
#@title Download transfer learning model
from google.colab import files
model.save('transfer_HLA.pt')
files.download(f'transfer_HLA.tsv')